As we have the data of the two tables grouped by the jurisdiction, we can now merge the two tables by the jurisdiction. If there is more than one carbonprice the AVG is comuted. 

In [2]:
import sqlite3
import datetime

conn = sqlite3.connect("../data/data.sqlite")
cursor = conn.cursor()

cursor.execute("Select * from emissions;")
sql_result_emmissions = cursor.fetchall()

current_year = datetime.datetime.now().year
avg_year_parts = []
for year in range(1990, current_year + 1):
    avg_year_parts.append(f"AVG(\"{year}\")")
avg_year_query_part = ", ".join(avg_year_parts)
select_statement = f"\"Jurisdiction Covered\", Region, \"Start Date\", {avg_year_query_part}"
sql_query = f"SELECT {select_statement} FROM carbonprice GROUP BY \"Jurisdiction Covered\";"
cursor.execute(sql_query)
sql_result_carbonprice = cursor.fetchall()
conn.close()

Therefor we need to find the common jurisdictions in both tables.

In [3]:
emmissions_jurisdictions = set([row[1] for row in sql_result_emmissions])
carbonprice_jurisdictions = set([row[0] for row in sql_result_carbonprice])
common_jurisdictions = emmissions_jurisdictions.intersection(carbonprice_jurisdictions)

result_emmissions = [row for row in sql_result_emmissions if row[1] in common_jurisdictions]
result_carbonprice = [row for row in sql_result_carbonprice if row[0] in common_jurisdictions]

With the common jurisdictions we can now merge the data of the two tables. We will merge the data by the jurisdiction and the year. So that the data is in format:
(jurisdiction, year, emissions_data, carbonprice_data)

In [4]:
merged_data = []
for jurisdiction in common_jurisdictions:
    jurisdiction_emissions = [row for row in result_emmissions if row[1] == jurisdiction]
    jurisdiction_carbonprice = [row for row in result_carbonprice if row[0] == jurisdiction]
    emissions_data_juri = jurisdiction_emissions[0]
    carbonprice_data_juri = jurisdiction_carbonprice[0]
    merged_data_perCountry = []
    for year_index, year in enumerate(range(1990, current_year + 1)):
        emissions_index = year - 1970 + 2
        carbonprice_index = year_index + 4

        result_value = (jurisdiction, year,
                        emissions_data_juri[emissions_index] if emissions_index < len(emissions_data_juri) else None,
                        carbonprice_data_juri[carbonprice_index] if carbonprice_index < len(carbonprice_data_juri) else None)
        merged_data_perCountry.append(result_value)
    merged_data.append(merged_data_perCountry)


merged_data = [[entry for entry in country_data if entry[2] is not None and entry[3] is not None] for country_data in merged_data]
merged_data = [country_data for country_data in merged_data if len(country_data) >= 1]

Now we can check for a correlation between the carbonprice and the emissions.

In [5]:
from scipy.stats import pearsonr
pearsonr_results = []
for country_data in merged_data:
    emissions = [row[2] for row in country_data]
    carbon_prices = [row[3] for row in country_data]
    if len(emissions) < 2 or len(carbon_prices) < 2:
        continue
    correlation_coefficient, p_value = pearsonr(emissions, carbon_prices)

    pearsonr_results.append((country_data[0][0], correlation_coefficient, p_value))

C:\Users\Felix\AppData\Local\Temp\ipykernel_13144\1422593431.py:8: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlation_coefficient, p_value = pearsonr(emissions, carbon_prices)


These are the results of the correlation.

In [7]:
import numpy as np
pearsonr_results = [row for row in pearsonr_results if not np.isnan(row[1]) and not np.isnan(row[2])]
max_correlation_coefficient = max([row[1] for row in pearsonr_results])
min_correlation_coefficient = min([row[1] for row in pearsonr_results])
avg_correlation_coefficient = np.mean([row[1] for row in pearsonr_results])
std_correlation_coefficient = np.std([row[1] for row in pearsonr_results])

max_p_value = max([row[2] for row in pearsonr_results])
min_p_value = min([row[2] for row in pearsonr_results])
avg_p_value = np.mean([row[2] for row in pearsonr_results])
std_p_value = np.std([row[2] for row in pearsonr_results])

print(f"Max correlation coefficient: {max_correlation_coefficient}")
print(f"Min correlation coefficient: {min_correlation_coefficient}")
print(f"Avg correlation coefficient: {avg_correlation_coefficient}")
print(f"Std correlation coefficient: {std_correlation_coefficient}")

print(f"Max p value: {max_p_value}")
print(f"Min p value: {min_p_value}")
print(f"Avg p value: {avg_p_value}")
print(f"Std p value: {std_p_value}")

Max correlation coefficient: 1.0
Min correlation coefficient: -1.0
Avg correlation coefficient: -0.35327103045626107
Std correlation coefficient: 0.5674591573927533
Max p value: 1.0
Min p value: 4.378645316693223e-10
Avg p value: 0.24655524910583237
Std p value: 0.3139993556109486


Now we can visualize the data.

In [ ]:
import matplotlib.pyplot as plt
# draw a plot of the results
plt.scatter([row[1] for row in pearsonr_results], [row[2] for row in pearsonr_results])
plt.xlabel("correlation coefficient")
plt.ylabel("p-value")
plt.title("Analyse of the correlation between emissions and carbon prices")
plt.show()

# draw a plot of the top ten emissions countries with carbon prices
emissions_sum = []
for row in result_emmissions:
    emissions_sum.append((row[1], sum(row[2:])))
latest_emissions_per_country = {}
sorted_emissions = sorted(emissions_sum, key=lambda x: x[1], reverse=True)
top_ten_emitting_countries = sorted_emissions[:10]
top_ten_emitting_countries_carbonPrice = []
top_ten_emitting_countries_emissions = []
for country in top_ten_emitting_countries:
    for row in result_carbonprice:
        if row[0] == country[0]:
            top_ten_emitting_countries_carbonPrice.append(row)
            break
    for row in result_emmissions:
        if row[1] == country[0]:
            top_ten_emitting_countries_emissions.append(row)
            break


years = list(range(1970, 2022))
for row in top_ten_emitting_countries_emissions:
    plt.plot(years, row[3:], label=row[1])
plt.title('Total Emissions of the top-ten emitting \n Jurisdictions over the years')
plt.xlabel('Year')
plt.ylabel('Total Emissions')
plt.legend(title='Jurisdiction', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

years = list(range(1990, current_year))
for row in top_ten_emitting_countries_carbonPrice:
    plt.plot(years, row[4:], label=row[0])
plt.title('Average Carbon Price of the top-ten emitting \n Jurisdictions with carbon price over the years')
plt.xlabel('Year')
plt.ylabel('Carbon Price')
plt.legend(title='Jurisdiction', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()